In [ ]:
from fastai.basics import *
from fastai.gen_doc.nbdoc import *

In [ ]:
import fastai
from fastai.version import __version__
print(__version__)

In [ ]:
from fastai.vision import *

In [ ]:
from fastai.tabular import *

y_pred = np.array(get_preds(learn.model, data.test_dl)[0]).reshape(-1,)
y_true = np.array(data.test_ds.tensors[1]).reshape(-1)

In [ ]:

df = pd.read_csv('../input/carpriceprediction/CarPrice_Assignment.csv')
dep_var = 'price'
cat_names = ['CarName','fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','enginetype','cylindernumber','fuelsystem']
cont_names = ['car_ID','symboling','wheelbase','carlength','carwidth','carheight', 'curbweight','enginesize','boreratio','stroke', 'compressionratio',
              'horsepower','peakrpm','citympg','highwaympg']
procs = [Categorify, Normalize]

In [ ]:
#Start index for creating a validation set from train_data
start_indx = len(train_data) - int(len(train_data) * 0.2)

In [ ]:
#End index for creating a validation set from train_data
end_indx = len(train_data)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#TabularList for Validation
val = (TabularList.from_df(train_data.iloc[start_indx:end_indx].copy(), path=path, cat_names=cat_names, cont_names=cont_names))

test = (TabularList.from_df(test_data, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs))

#TabularList for training
data = (TabularList.from_df(train_data, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(start_indx,end_indx)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [ ]:
#Display the data batch
data.show_batch(rows = 10)

In [ ]:
#Initializing the network
learn = tabular_learner(data, layers=[300,200, 100, 50], metrics= [rmse,r2_score])

In [ ]:
#Exploring the learning rates
learn.lr_find(start_lr = 1e-05,end_lr = 1e+05, num_it = 100)
learn.recorder.plot()

In [ ]:
#Fitting data and training the network
learn.fit_one_cycle(25)

In [ ]:
learn.show_results()

In [ ]:
#Display Predictions On Training Data
learn.show_results(ds_type=DatasetType.Train,rows = 5)
#Display Predictions On Validation Data
learn.show_results(ds_type=DatasetType.Valid)

In [ ]:
#Getting The Training And Validation Errors
tr = learn.validate(learn.data.train_dl)
va = learn.validate(learn.data.valid_dl)
print("The Metrics used In Evaluating The Network:", str(learn.metrics))
print("\nThe calculated RMSE & R-Squared For The Training Set :", tr[1:])
print("\nThe calculated RMSE & R-Squared For The Validation Set :", va[1:])

In [ ]:
#Plotting The losses for training and validation
learn.recorder.plot_losses()